## Qwen3-14B's Fine-Tuning Test

official
> ref: https://docs.unsloth.ai/models/qwen3-how-to-run-and-fine-tune#fine-tuning-qwen3-with-unsloth \
> ref: https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Qwen3_(14B)-Reasoning-Conversational.ipynb

note
> ref: https://note.com/sachi2222/n/na7b1d91ffb5d

In [1]:
# imports
## LLM
from unsloth import FastLanguageModel, is_bfloat16_supported
from trl import SFTTrainer, SFTConfig

import torch
from transformers import TrainingArguments

## Dataset
from datasets import load_dataset, Dataset
from unsloth import to_sharegpt
import re

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


## Load & Setting Model

In [ ]:
# model loading
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Qwen3-14B-unsloth-bnb-4bit",
    max_seq_length=40960, # context length
    load_in_4bit=True, # less mem mode
    load_in_8bit=False, # more mem mode
    full_finetuning=False # if u need full-FT
)

==((====))==  Unsloth 2025.9.11: Fast Qwen3 patching. Transformers: 4.56.2.
   \\   /|    NVIDIA RTX A6000. Num GPUs = 1. Max memory: 47.431 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
# select LoRA option
model = FastLanguageModel.get_peft_model(
    model,
    r=32, # Choose any num like 8, 16, 32, 64, 128
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
    lora_alpha=32, # alpha = r | r*2
    lora_dropout=0, # 0 is optimized
    bias="none", # "none" is optimized
    use_gradient_checkpointing="unsloth", # True | "unsloth", for too-long context
    random_state=3407,
    use_rslora=False, # under here, optical
    loftq_config=None
)

Unsloth 2025.9.11 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


## Convert Datasets

In [ ]:
# make Datasets
# load json
raw_data = load_dataset("json", data_files="./rspec_rules.json")

# convert hugginface-style
huggingface_data = Dataset.from_list(raw_data['train'])

# convert share_gpt-style
# Three dialogue sessions are preferred
share_data = to_sharegpt(huggingface_data,
                        merged_prompt="{instruction}",
                        merged_column_name="title",
                        output_column_name="description",
                        conversation_extension=3)


Merging columns:   0%|          | 0/23370 [00:00<?, ? examples/s]

Converting to ShareGPT:   0%|          | 0/23370 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/23370 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/23370 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/23370 [00:00<?, ? examples/s]

Extending conversations:   0%|          | 0/23370 [00:00<?, ? examples/s]

In [5]:
# Minor format changes
converted_share_data = [[{
            "role": "user" if message["from"] == "human" else "assistant",
            "content": re.sub(r"\('(.+?)',\)", r"\1", message["value"])
        }
        for message in item["conversations"]
    ]
    for item in share_data
]

In [6]:
converted_share_data[0]

[{'role': 'user', 'content': 'クロスサイトスクリプティングの脆弱性の例を教えて'},
 {'role': 'assistant',
  'content': 'Iqbolshoh Ilhomjonov の PHP Education Management におけるクロスサイトスクリプティングの脆弱性が起きているようです。PHP Education Manager v1.0 is vulnerable to Cross Site Scripting (XSS) in the worksheet.php file via the participant_name parameter. 該当するのはIqbolshoh Ilhomjonov\nPHP Education Management 1.0とのことです。'},
 {'role': 'user', 'content': '認証の欠如に関する脆弱性の例を教えて'},
 {'role': 'assistant',
  'content': 'wpeasypay の WordPress 用 wp easypay における認証の欠如に関する脆弱性が観測されている。The WP EasyPay – Square for WordPress plugin for WordPress is vulnerable to unauthorized modification of datadue to a missing capability check on the wpep_square_disconnect() function in all versions up to, and including, 4.2.3. This makes it possible for unauthenticated attackers to disconnect square. 影響するソフトウェアはwpeasypay\nwp easypay 4.2.4 未満である。'},
 {'role': 'user', 'content': 'クロスサイトスクリプティングの脆弱性の例を教えて'},
 {'role': 'assistant',
  'content': "Pencilwp の WordPress 用 X Ad

In [7]:
# Addition of <\llm_start\>, etc
conversations = tokenizer.apply_chat_template(
    converted_share_data,
    tokenize = False,
)

In [8]:
# Converted to Hugging Face format

targetdataset = Dataset.from_dict({"text": conversations})

## Train settings

In [9]:
# Train
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=targetdataset,
    eval_dataset=None, # test Datasets
    args=SFTConfig(
        dataset_text_field="text",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        # num_train_epochs=2,
        max_steps=100,
        learning_rate=2e-5,
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.001,
        lr_scheduler_type="linear",
        seed=3407,
        report_to="none"
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=24):   0%|          | 0/23370 [00:00<?, ? examples/s]

## Start FT

In [10]:
trainer_stats = trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 23,370 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 128,450,560 of 14,896,757,760 (0.86% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.621900
2,2.631800
3,2.639800
4,2.628300
5,2.546200
6,2.665300
7,2.608800
8,2.571400
9,2.426400
10,2.578300


## Inference

In [11]:
# Inference test
prompt="""
Please provide examples of vulnerabilities related to external control of filenames or pathnames.
"""

messages = [{"role" : "user", "content" : prompt}]

text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = False, # Disable thinking
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 2048, # Increase for longer outputs!
    temperature = 0.7, top_p = 0.8, top_k = 20, # For non thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)


Vulnerabilities related to **external control of filenames or pathnames** occur when an application uses user-supplied input to construct file paths or filenames without proper validation or sanitization. This can lead to serious security issues such as **Path Traversal**, **File Inclusion**, **Local File Inclusion (LFI)**, **Remote File Inclusion (RFI)**, **Directory Traversal**, and **Path Manipulation** attacks.

Below are several **examples of such vulnerabilities** and how they can be exploited.

---

## 🔍 Example 1: Path Traversal via File Download

### Vulnerable Code (PHP):
```php
$file = $_GET['file'];
readfile($file . ".txt");
```

### Exploitation:
```
http://example.com/download.php?file=../../etc/passwd
```

### Impact:
- The application reads and returns the contents of `/etc/passwd`, which is a sensitive file.

---

## 🔍 Example 2: Local File Inclusion (LFI)

### Vulnerable Code (PHP):
```php
include($_GET['page'] . ".php");
```

### Exploitation:
```
http://example.com/

In [12]:
# Inference test2
prompt="""
You are a white hat hacker tasked with discovering vulnerabilities in the provided source code.
Perform the following three actions on the source code below:
1. Identify the vulnerability
2. Present the risks of leaving it unaddressed
3. Provide a solution to eliminate the vulnerability

Ensure your output adheres to the following three points:
1. Output in Japanese
2. Be clear and concise
3. Use Markdown format

The source code is shown below:
---
---
<?php
// php_d100_roller.php
session_start();

// Keep roll history in session (max 100 entries)
if (!isset($_SESSION['d100_history'])) {
    $_SESSION['d100_history'] = [];
}

$errors = [];
$results = [];
$total = 0;
$count = 1;

if ($_SERVER['REQUEST_METHOD'] === 'POST') {
    // Get roll count (clamped between 1–100)
    $count = isset($_POST['count']) ? intval($_POST['count']) : 1;
    if ($count < 1) $count = 1;
    if ($count > 100) $count = 100;

    // Optional label
    $label = isset($_POST['label']) ? trim($_POST['label']) : '';

    // Perform rolls
    for ($i = 0; $i < $count; $i++) {
        $roll = random_int(1, 100);
        $results[] = $roll;
        $total += $roll;
    }

    // Add to history (newest first)
    $entry = [
        'time' => date('Y-m-d H:i:s'),
        'count' => $count,
        'label' => $label,
        'results' => $results,
        'total' => $total,
    ];

    array_unshift($_SESSION['d100_history'], $entry);
    if (count($_SESSION['d100_history']) > 100) {
        $_SESSION['d100_history'] = array_slice($_SESSION['d100_history'], 0, 100);
    }
}

$history = $_SESSION['d100_history'];
?>
<!doctype html>
<html lang="en">
<head>
<meta charset="utf-8">
<meta name="viewport" content="width=device-width,initial-scale=1">
<title>100-sided Dice Roller</title>
<style>
    body { font-family: system-ui, -apple-system, "Segoe UI", Roboto, "Helvetica Neue", Arial; padding: 20px; }
    .box { max-width: 820px; margin: 0 auto; }
    input[type="number"] { width: 80px; }
    .badge { display:inline-block; padding:6px 10px; margin:4px; border-radius:6px; background:#eee; }
    .roll { font-weight:700; }
    .history { margin-top:20px; }
    .card { padding:12px; border:1px solid #ddd; border-radius:8px; margin-bottom:12px; }
    .muted { color:#666; font-size:0.9rem; }
</style>
</head>
<body>
<div class="box">
    <h1>100-sided Dice Roller</h1>
    <form method="post" action="<?php echo htmlspecialchars($_SERVER['PHP_SELF']); ?>">
    <label>Number of rolls (1–100): <input type="number" name="count" value="<?php echo htmlspecialchars($count); ?>" min="1" max="100"></label>
    &nbsp;
    <label>Label (optional): <input type="text" name="label" value=""></label>
    &nbsp;
    <button type="submit">Roll</button>
    </form>

    <?php if (!empty($results)): ?>
    <div class="card">
        <div class="muted">Timestamp: <?php echo htmlspecialchars($entry['time']); ?></div>
        <h2>Results</h2>
        <div>
        <?php foreach ($results as $i => $r): ?>
            <span class="badge roll">#<?php echo $i+1; ?>: <?php echo $r; ?></span>
        <?php endforeach; ?>
        </div>
        <p>Total: <strong><?php echo $total; ?></strong> / Average: <strong><?php echo count($results) ? round($total / count($results), 2) : 0; ?></strong></p>
        <?php if ($entry['label'] !== ''): ?><p>Label: <?php echo htmlspecialchars($entry['label']); ?></p><?php endif; ?>
    </div>
    <?php endif; ?>

    <div class="history">
    <h2>History (last <?php echo count($history); ?> rolls)</h2>
    <?php if (empty($history)): ?>
        <p class="muted">No rolls yet.</p>
    <?php else: ?>
        <?php foreach ($history as $idx => $h): ?>
        <div class="card">
            <div class="muted"><?php echo htmlspecialchars($h['time']); ?> — Rolls: <?php echo $h['count']; ?><?php if ($h['label'] !== ''): ?> — Label: <?php echo htmlspecialchars($h['label']); ?><?php endif; ?></div>
            <div style="margin-top:8px;">
            <?php foreach ($h['results'] as $i => $r): ?>
                <span class="badge"><?php echo $r; ?></span>
            <?php endforeach; ?>
            </div>
            <p style="margin-top:8px;">Total: <?php echo $h['total']; ?> / Average: <?php echo count($h['results']) ? round($h['total'] / count($h['results']), 2) : 0; ?></p>
        </div>
        <?php endforeach; ?>
    <?php endif; ?>
    </div>

    <div style="margin-top:20px;" class="muted">
    <p>Note: Uses <code>random_int(1, 100)</code> for cryptographically secure random number generation. The session keeps up to 100 history entries.</p>
    </div>
</div>
</body>
</html>
"""

messages = [{"role" : "user", "content" : prompt}]

text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = False, # Disable thinking
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 4096, # Increase for longer outputs!
    temperature = 0.7, top_p = 0.8, top_k = 20, # For non thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)


# ソースコードの脆弱性分析

## 1. 脆弱性の特定

このソースコードには**セッションの注入**という脆弱性があります。`$_SESSION['d100_history']`にデータを保存する際、セッションIDが悪意のあるユーザーによって操作される可能性があるため、セッションハイジャックやセッション固定の脆弱性が発生するリスクがあります。

## 2. 無視した場合のリスク

- **セッションハイジャック**: 攻撃者がセッションIDを取得することで、他のユーザーのアクティビティを偽装して操作する可能性があります。
- **セッション固定**: 攻撃者がセッションIDを固定することで、認証をバイパスする可能性があります。
- **データの不正操作**: ロール履歴のデータが改ざんされる可能性があります。

## 3. 解決策

セッションのセキュリティを向上させるために、以下のような対策を講じます。

- **セッションIDの再生成**: ログインや重要なアクションの後にセッションIDを再生成します。
- **セッションのセキュリティ設定**: `session.cookie_secure`や`session.cookie_httponly`などの設定を有効にします。
- **セッションの有効期限を設定**: `session.gc_maxlifetime`でセッションの有効期限を設定し、長期間のセッションの継続を防ぎます。

---

### 修正後のコード例（セッションID再生成）

```php
session_start();

// セッションIDを再生成
session_regenerate_id(true);

// 以下は元のコード
if (!isset($_SESSION['d100_history'])) {
    $_SESSION['d100_history'] = [];
}
```

このように対策を講じることで、セッションに関する脆弱性を大幅に軽減できます。<|im_end|>


In [13]:
# Inference test3
prompt="""
You are a white hat hacker tasked with discovering vulnerabilities in the provided source code.
Perform the following three actions on the source code below:
1. Identify the vulnerability
2. Present the risks of leaving it unaddressed
3. Provide a solution to eliminate the vulnerability

Ensure your output adheres to the following three points:
1. Output in Japanese
2. Be clear and concise
3. Use Markdown format

The source code is shown below:
--- index.php
<?php
// index.php
session_start();

// load maze data
$mazes = include __DIR__ . '/maze_data.php';

// choose maze id (固定で sample1 を使うが、選択UIを後で追加可能)
$maze_id = 'sample1';

if (!isset($mazes[$maze_id])) {
    echo "Maze not found.";
    exit;
}

$maze = $mazes[$maze_id];

// initialize session if new or restart requested
if (!isset($_SESSION['maze']) || ($_SESSION['maze_id'] ?? '') !== $maze_id || isset($_GET['restart'])) {
    // store maze as array of strings
    $_SESSION['maze'] = $maze;
    $_SESSION['maze_id'] = $maze_id;
    $_SESSION['steps'] = 0;
    $_SESSION['start_time'] = time();
    // find start position S
    $start = null;
    foreach ($_SESSION['maze'] as $r => $row) {
        $c = strpos($row, 'S');
        if ($c !== false) {
            $start = ['r' => $r, 'c' => $c];
            // replace 'S' with space in stored maze so server treats it as open cell
            $_SESSION['maze'][$r] = substr_replace($_SESSION['maze'][$r], ' ', $c, 1);
            break;
        }
    }
    if (!$start) {
        echo "Start position not found in maze data.";
        exit;
    }
    $_SESSION['player'] = $start;
    // find goal (for convenience)
    $goal = null;
    foreach ($_SESSION['maze'] as $r => $row) {
        $c = strpos($row, 'G');
        if ($c !== false) {
            $goal = ['r' => $r, 'c' => $c];
            // replace 'G' with 'G' left (we keep it for rendering)
            break;
        }
    }
    $_SESSION['goal'] = $goal;
}

// JSON-encode maze for client-side rendering
$maze_json = json_encode($_SESSION['maze'], JSON_HEX_TAG|JSON_HEX_APOS|JSON_HEX_QUOT|JSON_HEX_AMP);
$player_json = json_encode($_SESSION['player']);
$goal_json = json_encode($_SESSION['goal']);
$start_time = $_SESSION['start_time'];
$steps = $_SESSION['steps'];
?>
<!doctype html>
<html lang="ja">
<head>
<meta charset="utf-8">
<meta name="viewport" content="width=device-width,initial-scale=1">
<title>PHP 2D 迷路ゲーム</title>
<link rel="stylesheet" href="style.css">
</head>
<body>
<div class="container">
  <h1>2D 迷路ゲーム</h1>
  <div class="controls">
    <button id="up">↑</button>
    <div>
      <button id="left">←</button>
      <button id="down">↓</button>
      <button id="right">→</button>
    </div>
    <button id="restart">Restart</button>
  </div>

  <div id="status">
    <span>ステップ: <span id="steps"><?php echo htmlspecialchars($steps, ENT_QUOTES); ?></span></span>
    <span>経過: <span id="elapsed">0s</span></span>
  </div>

  <div id="maze"></div>

  <div id="message" class="message"></div>
</div>

<script>
const maze = <?php echo $maze_json; ?>;
let player = <?php echo $player_json; ?>;
const goal = <?php echo $goal_json; ?>;
let startTime = <?php echo json_encode($start_time); ?>;
let steps = <?php echo json_encode($steps); ?>;

const mazeEl = document.getElementById('maze');
const messageEl = document.getElementById('message');
const stepsEl = document.getElementById('steps');
const elapsedEl = document.getElementById('elapsed');

function renderMaze() {
  // create grid
  mazeEl.innerHTML = '';
  const rows = maze.length;
  const cols = maze[0].length;
  const table = document.createElement('div');
  table.className = 'grid';
  table.style.gridTemplateColumns = `repeat(${cols}, 32px)`;
  for (let r = 0; r < rows; r++) {
    for (let c = 0; c < cols; c++) {
      const ch = maze[r].charAt(c);
      const cell = document.createElement('div');
      cell.className = 'cell';
      if (ch === '#') {
        cell.classList.add('wall');
      } else if (r === goal.r && c === goal.c) {
        cell.classList.add('goal');
        cell.textContent = 'G';
      } else {
        // empty
      }
      if (r === player.r && c === player.c) {
        const p = document.createElement('div');
        p.className = 'player';
        p.textContent = 'P';
        cell.appendChild(p);
      }
      table.appendChild(cell);
    }
  }
  mazeEl.appendChild(table);
}

function setMessage(msg, isError = false) {
  messageEl.textContent = msg;
  messageEl.className = isError ? 'message error' : 'message';
}

// ask server to move; server returns updated player pos and status
async function sendMove(direction) {
  try {
    const res = await fetch('move.php', {
      method: 'POST',
      headers: {'Content-Type': 'application/json'},
      body: JSON.stringify({dir: direction})
    });
    if (!res.ok) {
      setMessage('通信エラー: ' + res.status, true);
      return;
    }
    const data = await res.json();
    if (data.error) {
      setMessage('エラー: ' + data.error, true);
      return;
    }
    // update local state
    player = data.player;
    steps = data.steps;
    stepsEl.textContent = steps;
    renderMaze();
    if (data.cleared) {
      const sec = data.elapsed;
      setMessage(`クリア！ ステップ ${steps}、経過時間 ${sec} 秒`);
    } else {
      setMessage('');
    }
  } catch (e) {
    setMessage('例外: ' + e.message, true);
  }
}

// keyboard control
window.addEventListener('keydown', (ev) => {
  if (ev.key === 'ArrowUp') { ev.preventDefault(); sendMove('up'); }
  if (ev.key === 'ArrowDown') { ev.preventDefault(); sendMove('down'); }
  if (ev.key === 'ArrowLeft') { ev.preventDefault(); sendMove('left'); }
  if (ev.key === 'ArrowRight') { ev.preventDefault(); sendMove('right'); }
});

// buttons
document.getElementById('up').addEventListener('click', () => sendMove('up'));
document.getElementById('down').addEventListener('click', () => sendMove('down'));
document.getElementById('left').addEventListener('click', () => sendMove('left'));
document.getElementById('right').addEventListener('click', () => sendMove('right'));

document.getElementById('restart').addEventListener('click', () => {
  // reload with restart param to reset session
  window.location.href = window.location.pathname + '?restart=1';
});

// elapsed timer
function updateElapsed() {
  const now = Math.floor(Date.now() / 1000);
  const elapsed = now - startTime;
  elapsedEl.textContent = elapsed + 's';
}
setInterval(updateElapsed, 1000);

// initial render
renderMaze();
updateElapsed();
</script>
</body>
</html>

--- move.php
<?php
// move.php
session_start();
header('Content-Type: application/json; charset=utf-8');

// helper to respond
function json_err($msg, $code = 400) {
    http_response_code($code);
    echo json_encode(['error' => $msg]);
    exit;
}

// validate session
if (!isset($_SESSION['maze']) || !isset($_SESSION['player']) || !isset($_SESSION['goal'])) {
    json_err('セッション情報がありません。ゲームを再開してください。', 440);
}

// parse input
$raw = file_get_contents('php://input');
$data = json_decode($raw, true);
if (!is_array($data) || !isset($data['dir'])) {
    json_err('無効なリクエスト。', 400);
}

$dir = $data['dir'];
$allowed = ['up','down','left','right'];
if (!in_array($dir, $allowed, true)) {
    json_err('無効な方向指定。', 400);
}

// current state
$maze = $_SESSION['maze'];
$player = $_SESSION['player'];
$rows = count($maze);
$cols = strlen($maze[0]);

$dr = 0; $dc = 0;
switch ($dir) {
    case 'up': $dr = -1; break;
    case 'down': $dr = 1; break;
    case 'left': $dc = -1; break;
    case 'right': $dc = 1; break;
}

// target
$nr = $player['r'] + $dr;
$nc = $player['c'] + $dc;

// bounds check
if ($nr < 0 || $nr >= $rows || $nc < 0 || $nc >= $cols) {
    // invalid move: ignore
    echo json_encode([
        'player' => $player,
        'steps' => $_SESSION['steps'],
        'cleared' => false,
        'message' => '境界外の移動',
    ]);
    exit;
}

// check wall
$ch = $maze[$nr][$nc] ?? '#';
if ($ch === '#') {
    // wall: ignore move
    echo json_encode([
        'player' => $player,
        'steps' => $_SESSION['steps'],
        'cleared' => false,
        'message' => '壁です',
    ]);
    exit;
}

// move allowed
$_SESSION['player'] = ['r' => $nr, 'c' => $nc];
$_SESSION['steps'] = ($_SESSION['steps'] ?? 0) + 1;

// goal check (maze cell might contain 'G')
$cleared = false;
$goal = $_SESSION['goal'];
if ($nr === $goal['r'] && $nc === $goal['c']) {
    $cleared = true;
}

$now = time();
$elapsed = $now - ($_SESSION['start_time'] ?? $now);

echo json_encode([
    'player' => $_SESSION['player'],
    'steps' => $_SESSION['steps'],
    'cleared' => $cleared,
    'elapsed' => $elapsed,
]);
exit;

---  maze_data.php
<?php
// 迷路データを配列で返す（参照 / include して使う）
// 文字: '#' 壁, ' ' 通路, 'S' スタート, 'G' ゴール

return [
    // maze id => array of rows (strings)
    'sample1' => [
        "#############",
        "#S    #     #",
        "# ##  # ### #",
        "#    ##   # #",
        "#### ###  # #",
        "#      #   G#",
        "# ###### ####",
        "#          ##",
        "#############"
    ],
];
"""

messages = [{"role" : "user", "content" : prompt}]

text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = False, # Disable thinking
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 4096, # Increase for longer outputs!
    temperature = 0.7, top_p = 0.8, top_k = 20, # For non thinking
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)


# 指摘された脆弱性とその対策

## 1. 脆弱性の特定

このコードには**セッション固定化の脆弱性**が存在します。  
`move.php` で使用される `$_SESSION['start_time']` が、`$_SESSION['start_time'] ?? $now` と指定されているため、セッションが初期化されていない場合に `$now`（現在時刻）が使用されます。この挙動により、**セッションを不正に操作する可能性がある**。

また、`$_SESSION['maze']` が未定義の場合は `#` とみなされるため、**セッションデータが破損した場合でもゲームが異常終了せずに実行される**という問題も存在します。

---

## 2. 脆弱性のリスク

- **セッション固定化の脆弱性**により、不正なユーザーが自分のセッションを他のユーザーに不正に利用する可能性があります。
- **セッションデータの破損**により、ゲームの動作が意図しない挙動を示す可能性があります。
- **セキュリティの信頼性が低下**し、アプリケーションの信頼性に悪影響を及ぼします。

---

## 3. 脆弱性の修正方法

### `move.php` の修正箇所

```php
// move.php 修正箇所
$cleared = false;
$goal = $_SESSION['goal'];
if ($nr === $goal['r'] && $nc === $goal['c']) {
    $cleared = true;
}

$now = time();
$elapsed = $now - ($_SESSION['start_time'] ?? $now);
```

### 修正内容

`$_SESSION['start_time']` が存在しない場合、`$now`（現在時刻）を代入する代わりに、**セッションが初期化されていない場合にエラーを返す**ように変更します。

### 修正コード例

```php
// move.php 修正後のコード
if (!isset($_SESSION['start_time'])) {
    json_err('セッションの開始時刻が未設定です。', 400);
}

$now = time();


## Save LoRA/Model

In [14]:
# save LoRA
model.save_pretrained("qwen3-14b_Vulnerability_Detection")
tokenizer.save_pretrained("qwen3-14b_Vulnerability_Detection")

KeyboardInterrupt: 

In [ ]:
# save ALL-MODEL
model.save_pretrained_merged("qwen3-14b_Vulnerability_Detection", tokenizer, save_method="merged_16bit")

In [ ]:
# save ALL-MODEL as .gguf
model.save_pretrained_merged("qwen3-14b_Vulnerability_Detection", tokenizer)

## Upload LoRA/Model

In [ ]:
# upload
